## Improve MNIST with Convolutions

Para este ejercicio, vea si puede mejorar MNIST al 99,5 % de precisión o más agregando solo una capa convolucional y una capa MaxPooling 2D al modelo de la asignación de la semana anterior.

Debe dejar de entrenar una vez que la precisión supere esta cantidad. Debería ocurrir en menos de 10 épocas, por lo que está bien codificar la cantidad de épocas para el entrenamiento, pero su entrenamiento debe terminar una vez que alcance la métrica anterior. Si no es así, deberá rediseñar su devolución de llamada.

Cuando se haya alcanzado el 99,5 % de precisión, debe imprimir la cadena "Se alcanzó el 99,5 % de precisión, por lo que se canceló el entrenamiento".

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

mnist = tf.keras.datasets.mnist

In [6]:
# Load the data

# Discard test set
(training_images, training_labels), _ =  mnist.load_data()
      


In [7]:
def reshape_and_normalize(images):

    # Reshape the images to add an extra dimension
    images = images.reshape([60000, 28, 28, 1])
    
    # Normalize pixel values
    images = images / 255.0

    return images

Un paso importante cuando se trata de datos de imágenes es preprocesar los datos. Durante el paso de preprocesamiento, puede aplicar transformaciones al conjunto de datos que se alimentará a su red neuronal convolucional.
Aquí aplicará dos transformaciones a los datos:


Remodele los datos para que tengan una dimensión adicional. La razón de esto es que normalmente utilizará matrices tridimensionales (sin contar la dimensión del lote) para representar datos de imagen. La tercera dimensión representa el color usando valores RGB. Estos datos pueden estar en formato blanco y negro, por lo que la tercera dimensión en realidad no agrega ninguna información adicional para el proceso de clasificación, pero es una buena práctica a pesar de todo.


Normalice los valores de píxel para que sean valores entre 0 y 1. Puede lograr esto dividiendo cada valor en la matriz por el máximo.
Recuerda que estos tensores son del tipo numpy.ndarray, por lo que puedes usar funciones como reformar o dividir para completar la siguiente función reformar_y_normalizar:

In [8]:

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")

Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


In [9]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') > 0.995:
                print("\nReached 99% accuracy so cancelling training!") 
                self.model.stop_training = True



In [10]:
# GRADED FUNCTION: convolutional_model
def convolutional_model():
    ### START CODE HERE

    # Define the model, it should have 5 layers:
    # - A Conv2D layer with 32 filters, a kernel_size of 3x3, ReLU activation function
    #    and an input shape that matches that of every image in the training set
    # - A MaxPooling2D layer with a pool_size of 2x2
    # - A Flatten layer with no arguments
    # - A Dense layer with 128 units and ReLU activation function
    # - A Dense layer with 10 units and softmax activation function
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ]) 

    ### END CODE HERE

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

In [11]:
# Save your untrained model
model = convolutional_model()

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.1391 - accuracy: 0.9585
Epoch 2/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0469 - accuracy: 0.9851
Epoch 3/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0321 - accuracy: 0.9900
Epoch 4/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0230 - accuracy: 0.9925
Epoch 5/10
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0178 - accuracy: 0.9944
Epoch 6/10
1873/1875 [============================>.] - ETA: 0s - loss: 0.0135 - accuracy: 0.9956
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 35s 19ms/step - loss: 0.0135 - accuracy: 0.9956


In [12]:
print(f"Your model was trained for {len(history.epoch)} epochs")

Your model was trained for 6 epochs
